In [17]:
import numpy as np
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler

import hdbscan
from hdbscan.validity import validity_index
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score

In [ ]:
events = pd.read_excel('events.xlsx', sheet_name=3)

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EVENT_DATE  10000 non-null  datetime64[ns]
 1   EVENT_NAME  10000 non-null  object        
 2   USER_ID     10000 non-null  int64         
 3   CHAPTER     4000 non-null   float64       
 4   RESULT      2000 non-null   float64       
 5   REASON      2614 non-null   object        
 6   SOURCE      1386 non-null   object        
 7   AMOUNT      1998 non-null   float64       
 8   GAME_MODE   3975 non-null   object        
 9   NAME        2000 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 781.4+ KB


In [5]:
events.head(50)

,EVENT_DATE,EVENT_NAME,USER_ID,CHAPTER,RESULT,REASON,SOURCE,AMOUNT,GAME_MODE,NAME
0,2023-09-03,CurrencyEarned,247259373,NaN,NaN,NaN,DailyFree,30.0,NaN,NaN
1,2023-09-03,ChapterProgress,298339933,3.0,0.0,NaN,NaN,NaN,Normal,NaN
2,2023-09-05,CurrencyEarned,246407236,NaN,NaN,NaN,Achievements,10.0,NaN,NaN
3,2023-09-05,ChapterStarted,297123727,18.0,NaN,NaN,NaN,NaN,Normal,NaN
4,2023-09-06,ChapterStarted,296425420,24.0,NaN,NaN,NaN,NaN,Hard,NaN
5,2023-09-06,InAppPurchaseCompleted,279337211,NaN,NaN,NaN,NaN,NaN,NaN,Gem_Tier3
6,2023-09-03,ChapterProgress,298331945,6.0,0.0,NaN,NaN,NaN,Normal,NaN
7,2023-09-05,ChapterStarted,288198480,45.0,NaN,NaN,NaN,NaN,Normal,NaN
8,2023-09-03,CurrencyEarned,250987687,NaN,NaN,NaN,ChapterRewards,20.0,NaN,NaN
9,2023-09-03,CurrencyEarned,244507802,NaN,NaN,NaN,GrowthFund,150.0,NaN,NaN


In [3]:
currency_data = events[events['EVENT_NAME'] == 'CurrencyEarned']
user_metrics = currency_data.groupby('USER_ID').agg(
    TotalCurrencyEarned=('AMOUNT', 'sum'),
    EarnFrequency=('EVENT_NAME', 'count'),
    UniqueSources=('SOURCE', 'nunique')
).reset_index()

In [78]:
user_metrics

,USER_ID,TotalCurrencyEarned,EarnFrequency,UniqueSources
0,240748227,80.0,3,2
1,242208421,20.0,1,1
2,242499511,110.0,4,2
3,242722793,70.0,2,1
4,242873382,460.0,12,6
...,...,...,...,...
107,252188172,60.0,2,1
108,252223904,135.0,6,5
109,252224148,20.0,1,1
110,252313885,140.0,3,2


In [4]:
currency_data = events[events['EVENT_NAME'] == 'CurrencyEarned']
# Step 1: Create source-level statistics for each user
source_stats = currency_data.groupby(['USER_ID', 'SOURCE']).agg(
    SourceAmount=('AMOUNT', 'sum'),
    SourceFrequency=('EVENT_NAME', 'count')
).reset_index()

# Step 2: Pivot to create features for each source (one-hot-like)
source_pivot = source_stats.pivot(index='USER_ID', columns='SOURCE', values='SourceAmount').fillna(0)


In [5]:
total_earned = source_pivot.sum(axis=1)
source_proportions = source_pivot.div(total_earned, axis=0).fillna(0)

In [6]:
user_metrics = user_metrics.merge(source_proportions, on='USER_ID', how='left')

# Cluster

In [9]:
features = user_metrics.drop(columns=['USER_ID'])  # Remove USER_ID
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Apply HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, metric='euclidean')
user_metrics['Cluster'] = clusterer.fit_predict(features_scaled)

# Cluster Analysis by Source

In [10]:
cluster_analysis = user_metrics.groupby('Cluster').agg(
    TotalCurrencyEarned=('TotalCurrencyEarned', 'mean'),
    EarnFrequency=('EarnFrequency', 'mean'),
).reset_index()

In [11]:
# Create subsets for each cluster
cluster_negative = user_metrics[user_metrics['Cluster'] == -1]
cluster_zero = user_metrics[user_metrics['Cluster'] == 0]
cluster_one = user_metrics[user_metrics['Cluster'] == 1]

# Visuals

In [12]:
fig = px.scatter(
    user_metrics, 
    x='TotalCurrencyEarned', 
    y='EarnFrequency', 
    color='Cluster',
    hover_data=user_metrics.columns,
    title="All Clusters Overview"
)

fig.update_layout(
    xaxis_title="Total Currency Earned",
    yaxis_title="Earn Frequency",
    legend_title="Cluster"
)

fig.show()

# DBCV

In [13]:
# Range of min_cluster_size values to test
min_cluster_sizes = range(2, 51, 2)  # From 5 to 50 in steps of 5
dbcv_scores = []

for size in min_cluster_sizes:
    clusterer = hdbscan.HDBSCAN(min_cluster_size=size, metric='euclidean')
    labels = clusterer.fit_predict(features_scaled)
    
    # Compute DBCV only if there are multiple clusters formed
    if len(set(labels)) > 1:  # Ensure we have meaningful clusters
        dbcv = validity_index(features_scaled, labels, metric='euclidean')
        dbcv_scores.append((size, dbcv))
    else:
        dbcv_scores.append((size, None))

# Find the best min_cluster_size
best_size, best_dbcv = max((s for s in dbcv_scores if s[1] is not None), key=lambda x: x[1])
print(f"Best min_cluster_size: {best_size} with DBCV score: {best_dbcv}")

Best min_cluster_size: 4 with DBCV score: 0.26500128452345306


In [14]:
# Create the line plot using Plotly
fig = go.Figure()

# Extract valid scores for plotting
valid_sizes = [size for size, dbcv in dbcv_scores if dbcv is not None]
valid_dbcvs = [dbcv for size, dbcv in dbcv_scores if dbcv is not None]


fig.add_trace(go.Scatter(
    x=valid_sizes,
    y=valid_dbcvs,
    mode='lines+markers',
    marker=dict(size=8, color='blue'),
    line=dict(width=2, color='blue'),
    name='DBCV Score'
))

# Add titles and axis labels
fig.update_layout(
    title="DBCV Score vs. min_cluster_size",
    xaxis=dict(title="min_cluster_size"),
    yaxis=dict(title="DBCV Score"),
    template="plotly_white",
    width=800,
    height=500
)

# Show the Plotly plot
fig.show()

In [15]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=best_size, metric='euclidean')
user_metrics['Cluster'] = clusterer.fit_predict(features_scaled)

Cluster 1 - Low Activity Users:

TotalCurrencyEarned: 20.0
EarnFrequency: 1 (This user earns currency infrequently, just once in the data).
UniqueSources: 1 (The user earned from a single source).
This cluster might represent low-engagement users or users who don't frequently interact with the game's currency earning features. They could be users who are new to the game or not very active.

Cluster 2 - Medium Activity Users:

TotalCurrencyEarned: 80.0 to 110.0
EarnFrequency: 3 to 4 (These users earn currency 3 to 4 times, indicating moderate interaction with the game).
UniqueSources: 2 (They earn from multiple sources).
This cluster likely represents moderate-engagement users. They are more involved in the game than those in Cluster 1, earning from multiple sources and participating in events with moderate frequency. They could be regular users who are interested in progressing through the game but may not be heavily invested.

Cluster 3 - High Activity Users:

These users would likely have high total currency earned, frequent earnings, and engagement with multiple sources.
We would expect them to be highly involved, possibly spending more time on the game, completing a variety of tasks, and engaging with both in-game activities and promotions.

In [90]:
user_metrics

,USER_ID,TotalCurrencyEarned,EarnFrequency,UniqueSources,Achievements,BattlePass_Tier0,BossIntroduction,ChapterRewards,CommunityPage,DailyFree,...,InGame,LevelUp,MonthlyDeal_Tier1,MonthlyDeal_Tier2,RedeemCode,ShopRoulette,StarterGem,ThreeDaysEvent,WeeklyQuest,Cluster
0,240748227,80.0,3,2,0.000000,0.0,0.000000,0.000000,0.0,0.750000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,242208421,20.0,1,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,242499511,110.0,4,2,0.000000,0.0,0.000000,0.000000,0.0,0.818182,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,242722793,70.0,2,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,242873382,460.0,12,6,0.065217,0.0,0.000000,0.065217,0.0,0.260870,...,0.000000,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,252188172,60.0,2,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
108,252223904,135.0,6,5,0.000000,0.0,0.000000,0.370370,0.0,0.222222,...,0.000000,0.148148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
109,252224148,20.0,1,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
110,252313885,140.0,3,2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [16]:
# Select only the columns that are relevant for aggregation (excluding non-numeric columns like 'USER_ID' and 'Cluster')
numeric_columns = user_metrics.select_dtypes(include=['float64', 'int64']).columns

# Remove 'USER_ID' and 'Cluster' from the list of columns to aggregate
columns_to_aggregate = [col for col in numeric_columns if col not in ['USER_ID', 'Cluster']]

# Perform the aggregation
cluster_analysis = user_metrics.groupby('Cluster')[columns_to_aggregate].agg('mean').reset_index()

print(cluster_analysis)

   Cluster  TotalCurrencyEarned  EarnFrequency  UniqueSources  Achievements  \
0       -1           500.384615      14.584615       5.446154      0.053805   
1        0          1130.200000      47.400000       9.600000      0.103643   
2        1            29.375000       1.125000       1.000000      0.000000   
3        2           166.923077       7.038462       3.615385      0.060215   

   BattlePass_Tier0  BossIntroduction  ChapterRewards  CommunityPage  \
0          0.009698          0.073930        0.118262       0.003389   
1          0.048325          0.022664        0.056147       0.000000   
2          0.000000          0.000000        0.000000       0.000000   
3          0.000000          0.026659        0.024668       0.000000   

   DailyFree  ...  GrowthFund    InGame   LevelUp  MonthlyDeal_Tier1  \
0   0.258689  ...    0.073537  0.009711  0.059443           0.006393   
1   0.405000  ...    0.017281  0.011175  0.059345           0.000000   
2   0.000000  ...    0.0000

Possible Interpretation:
* Cluster -1: Outliers or noise (high currency earned but low engagement), may need further investigation to understand why they don't follow typical engagement patterns.
* Cluster 0: Highly engaged users, likely to be your core users, those who are highly active and have significant in-game activity.
* Cluster 1: Casual users or those with minimal engagement. Consider these users as potential targets for re-engagement campaigns.
* Cluster 2: Intermediate users who are moderately engaged but could potentially be pushed towards higher engagement. They may require additional incentives or features to increase their activity levels.

Choosing Between K-means and HDBSCAN
Use K-means if:

* The data has spherical clusters of similar sizes.
* You know the number of clusters in advance.
* Speed and simplicity are priorities.

Use HDBSCAN if:

* Clusters are irregularly shaped or have varying densities.
* You need robustness against noise and outliers.
* The number of clusters is unknown or difficult to define.

# Visuals Of Cluster

In [ ]:
from plotly.subplots import make_subplots

# Create subplots (3 columns)
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['TotalCurrencyEarned', 'EarnFrequency', 'UniqueSources'],
    # shared_yaxes=True  # To make y-axes consistent across all plots
)

# Add bar plots for each metric
fig.add_trace(
    go.Bar(x=cluster_analysis['Cluster'], y=cluster_analysis['TotalCurrencyEarned'], name='TotalCurrencyEarned'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=cluster_analysis['Cluster'], y=cluster_analysis['EarnFrequency'], name='EarnFrequency'),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=cluster_analysis['Cluster'], y=cluster_analysis['UniqueSources'], name='UniqueSources'),
    row=1, col=3
)

# Update layout
fig.update_layout(
    title='Cluster Analysis of Key Metrics',
    showlegend=False,
    height=400,
    width=1200
)

fig.show()

# Gaussian Mixture and DBI Score

In [18]:
component_range = range(1, 11)
dbi_scores = []

for n_components in component_range:
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(features_scaled)
    labels = gmm.predict(features_scaled)
    
    # Skip DBI for single cluster (n_components=1)
    if n_components > 1:
        dbi = davies_bouldin_score(features_scaled, labels)
        dbi_scores.append((n_components, dbi))
    else:
        dbi_scores.append((n_components, None))

In [20]:
# Create the line plot using Plotly
fig = go.Figure()

# Extract valid scores for plotting
valid_components = [n for n, dbi in dbi_scores if dbi is not None]
valid_dbi = [dbi for n, dbi in dbi_scores if dbi is not None]


fig.add_trace(go.Scatter(
    x=valid_components,
    y=valid_dbi,
    mode='lines+markers',
    marker=dict(size=8, color='blue'),
    line=dict(width=2, color='blue'),
    name='DBI Score'
))

# Add titles and axis labels
fig.update_layout(
    title="DBI Score vs. min_cluster_size",
    xaxis=dict(title="min_cluster_size"),
    yaxis=dict(title="DBI Score"),
    template="plotly_white",
    width=800,
    height=500
)

# Show the Plotly plot
fig.show()

In [23]:
# Range of min_cluster_size to test
min_cluster_sizes = range(2, 15, 1)

# To store DBI scores for each min_cluster_size
dbi_scores = []

for size in min_cluster_sizes:
    clusterer = hdbscan.HDBSCAN(min_cluster_size=size, metric='euclidean')
    labels = clusterer.fit_predict(features_scaled)
    
    # Compute DBI only if meaningful clusters exist
    if len(set(labels)) > 1 and -1 in labels:  # Ensure there are both noise and clusters
        score = davies_bouldin_score(features_scaled[labels != -1], labels[labels != -1])
        dbi_scores.append((size, score))
    else:
        dbi_scores.append((size, None))

# Extract valid scores for plotting
valid_sizes = [size for size, score in dbi_scores if score is not None]
valid_dbi_scores = [score for size, score in dbi_scores if score is not None]

# Find the best min_cluster_size based on DBI
best_size = valid_sizes[np.argmin(valid_dbi_scores)]
best_score = min(valid_dbi_scores)

print(f"Best min_cluster_size: {best_size} with DBI score: {best_score}")

Best min_cluster_size: 10 with DBI score: 0.2657884507754396
